In [7]:
#Download data tiny shakespeeare
!curl https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt >> input.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1089k  100 1089k    0     0  1001k      0  0:00:01  0:00:01 --:--:-- 1002k


In [8]:
# Read it and inspect it 
with open("input.txt", 'r', encoding="utf-8") as f:
    text = f.read()

In [10]:
print(f"Length of dataset is {len(text)}")

Length of dataset is 2230788


In [11]:
# Let's look at the fisrst 1000 charecters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [15]:
#Here are all unique charecters that occur in the text 
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(f"The unique charecters are: {''.join(chars)}")
print(f"Total number of charecters {vocab_size}")

The unique charecters are: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
Total number of charecters 65


In [23]:
# Create a mapping from charecter to integers
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encoder = lambda s: [stoi[c] for c in s] # encoder takes string of char and return a list of int
decoder = lambda l: ''.join([itos[i] for i in l]) # decoder takes a list of int and return a string of chars

print(f"Encode of 'Hi there' is { encoder('Hi There')}")
print(f"Decoded version of { encoder('Hi There')} is {decoder(encoder('Hi There'))}")
      


Encode of 'Hi there' is [20, 47, 1, 32, 46, 43, 56, 43]
Decoded version of [20, 47, 1, 32, 46, 43, 56, 43] is Hi There


In [25]:
# Let's now encode the entire text dataset and store it into a torc.Tensor 
import torch
data = torch.tensor(encoder(text), dtype=torch.long)
print(f"Shape of the data is {data.shape} and type of data is {data.type}")
print(f"Data example {data[:1000]}")

Shape of the data is torch.Size([2230788]) and type of data is <built-in method type of Tensor object at 0x10f911940>
Data example tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  

In [26]:
# Let's now split the dataset into train and validation sets
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]


In [27]:
block_size = 8 # context window
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [29]:
#JUst exploration of block size and input and terget
x = train_data[:block_size]
y = train_data[1: block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"When input is {context} target is {target}")

When input is tensor([18]) target is 47
When input is tensor([18, 47]) target is 56
When input is tensor([18, 47, 56]) target is 57
When input is tensor([18, 47, 56, 57]) target is 58
When input is tensor([18, 47, 56, 57, 58]) target is 1
When input is tensor([18, 47, 56, 57, 58,  1]) target is 15
When input is tensor([18, 47, 56, 57, 58,  1, 15]) target is 47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) target is 58


In [41]:
ix = torch.randint(len(data) - block_size, (4,))
ix

tensor([1489066,  771471,  498997,  465186])

In [45]:
torch.manual_seed(1337)
batch_size = 4 # Howmany independent squence will be processed in parallel 
block_size = 8 # What is the maximum context length of predictions?
def get_batch(split):
    #Generate a small batch of data of inputs x and target y
    data = train_data if  split =='train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i: i + block_size] for i in ix])
    y = torch.stack([data[i + 1 : i + block_size+1] for i in ix])
    return x, y

xb, yb = get_batch("train")
print('inputs: ')
print(xb.shape)
print(xb)
print("targets: ")
print(yb.shape)
print(yb)
print("----")

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"When input is {context.tolist()} the targe {target}")

    

inputs: 
torch.Size([4, 8])
tensor([[52, 43, 57, 57,  6,  1, 57, 53],
        [19, 53, 42,  1, 49, 43, 43, 54],
        [ 1, 39, 52, 42,  1, 53, 50, 42],
        [47, 43,  8,  0,  0, 31, 13, 25]])
targets: 
torch.Size([4, 8])
tensor([[43, 57, 57,  6,  1, 57, 53, 52],
        [53, 42,  1, 49, 43, 43, 54,  1],
        [39, 52, 42,  1, 53, 50, 42,  1],
        [43,  8,  0,  0, 31, 13, 25, 28]])
----
When input is [52] the targe 43
When input is [52, 43] the targe 57
When input is [52, 43, 57] the targe 57
When input is [52, 43, 57, 57] the targe 6
When input is [52, 43, 57, 57, 6] the targe 1
When input is [52, 43, 57, 57, 6, 1] the targe 57
When input is [52, 43, 57, 57, 6, 1, 57] the targe 53
When input is [52, 43, 57, 57, 6, 1, 57, 53] the targe 52
When input is [19] the targe 53
When input is [19, 53] the targe 42
When input is [19, 53, 42] the targe 1
When input is [19, 53, 42, 1] the targe 49
When input is [19, 53, 42, 1, 49] the targe 43
When input is [19, 53, 42, 1, 49, 43] the ta

In [46]:
print(xb)

tensor([[52, 43, 57, 57,  6,  1, 57, 53],
        [19, 53, 42,  1, 49, 43, 43, 54],
        [ 1, 39, 52, 42,  1, 53, 50, 42],
        [47, 43,  8,  0,  0, 31, 13, 25]])


In [66]:
import torch 
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)
class Bigramlanguagemodel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        #each token directly reads off the logits for the next token from a lookup table
        self.token_embdding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx and target are both (B,T) tensor of integer
        logits = self.token_embdding_table(idx) #(B,T,C)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view (B*T, C)
            targets = targets.view(-1)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the curren context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            logits = logits[:, -1, :] # becomes (B,C)
            # apply softmax to get probabilitis
            probs = F.softmax(logits, dim=-1) # (B,C)
            idx_next = torch.multinomial(probs, num_samples=1) # (B,1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim = 1) # ((B, T+1)
        return idx
        
m = Bigramlanguagemodel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)
idx = torch.zeros((1,1),  dtype=torch.long)
print(decoder(m.generate(idx, max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.5059, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [67]:
optimizer = torch.optim.Adam(m.parameters(), lr=1e-3)

In [72]:
batch_size = 32
for steps in range (10000):
    xb, yb = get_batch("train")
    #evaluate loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    print(loss.item())
    

2.333346128463745
2.444169282913208
2.497849225997925
2.4057507514953613
2.5626235008239746
2.4783122539520264
2.4412333965301514
2.3809523582458496
2.363809585571289
2.3401002883911133
2.449542760848999
2.4690585136413574
2.550978899002075
2.5012147426605225
2.3541958332061768
2.487748861312866
2.427421808242798
2.4641106128692627
2.4644241333007812
2.4734268188476562
2.4456698894500732
2.5789647102355957
2.4331436157226562
2.474790334701538
2.378185749053955
2.45542049407959
2.5481276512145996
2.4397084712982178
2.3160650730133057
2.4710044860839844
2.618291139602661
2.463441848754883
2.3827133178710938
2.4817557334899902
2.47005295753479
2.439293622970581
2.4110891819000244
2.518799304962158
2.366431951522827
2.2676198482513428
2.3972702026367188
2.5264928340911865
2.330796957015991
2.6259348392486572
2.3622000217437744
2.5419187545776367
2.4415745735168457
2.528834342956543
2.4749248027801514
2.500990629196167
2.4459517002105713
2.384559392929077
2.300508737564087
2.461017370223999

In [ ]:
print(decoder(m.generate(idx = torch.zeros((1,1), dtype=torch.long), max_new_tokens=500)[0].tolist()))

### The mathemathical trick in self attention

In [86]:
# consider the folloing toy example
torch.manual_seed(1337)
B, T, C = 4, 8, 2 # batch, time, channels
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [90]:
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] #(t,C)
        xbow[b,t] = torch.mean(xprev, 0)
        

In [91]:
x[0]


tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [102]:
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [106]:
wei = torch.tril(torch.ones(T,T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 =  wei @ x #(T,T) @ (B, T, C) ---> (B, T, C)

In [108]:
torch.allclose(xbow, xbow2)

True

In [111]:
tril  = torch.tril(torch.ones(T, T))
wei = torch.zeros(T, T)
wei = wei.masked_fill(tril==0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

True

In [97]:
torch.tril(torch.ones(3,3))

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [101]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
a = a / torch.sum(a, dim=1, keepdim=True)
b = torch.randint(0, 10, (3,2)).float()
c = a @ b
print('a=')
print(a)
print('----')
print('b=')
print(b)
print('----')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
----
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
----
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [112]:
torch.arange(8)

tensor([0, 1, 2, 3, 4, 5, 6, 7])

In [117]:
#version: 4
torch.manual_seed(1337)
B, T, C = 4, 8, 32 # batch, time, channel
x = torch.randn(B, T, C)

head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) #(B, T, 16)
q = query(x) #(B, T, 16) 

wei = q @ k.transpose(-2, -1) # (B, T , 16 ) @ (B, 16, T) -->  (B, T, T)


tril  = torch.tril(torch.ones(T, T))
#wei = torch.zeros(T, T)
wei = wei.masked_fill(tril==0, float('-inf'))
wei = F.softmax(wei, dim=-1)
v = value(x)
out = wei @ v
out.shape

torch.Size([4, 8, 16])

In [118]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

In [2]:
import torch
class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])